In [4]:
import warnings
warnings.filterwarnings('ignore')
# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
# Specify the number of binary digits. 
NUM_DIGITS = 10

In [7]:
## Setup the training data for 101-1024. 1024 is the highest number countable with 10 binary digits
raw_training_data = np.array(range(101, 2**NUM_DIGITS))

In [8]:
## This is a numpy array of integers.
raw_training_data[0:20]

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120])

In [9]:
# We need to binary encode our inputs
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [10]:
print(1, binary_encode(1, NUM_DIGITS))
print(2, binary_encode(2, NUM_DIGITS))
print(3, binary_encode(3, NUM_DIGITS))
print(4, binary_encode(4, NUM_DIGITS))

1 [1 0 0 0 0 0 0 0 0 0]
2 [0 1 0 0 0 0 0 0 0 0]
3 [1 1 0 0 0 0 0 0 0 0]
4 [0 0 1 0 0 0 0 0 0 0]


In [11]:
binary_encode(3, NUM_DIGITS)

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
print(256, binary_encode(256, NUM_DIGITS))
print(512, binary_encode(512, NUM_DIGITS))
print(1023, binary_encode(1023, NUM_DIGITS))

256 [0 0 0 0 0 0 0 0 1 0]
512 [0 0 0 0 0 0 0 0 0 1]
1023 [1 1 1 1 1 1 1 1 1 1]


In [13]:
encoded_training_data = [binary_encode(i, NUM_DIGITS) for i in raw_training_data]

In [14]:
x_train = np.array(encoded_training_data)
x_train[0]

array([1, 0, 1, 0, 0, 1, 1, 0, 0, 0])

In [15]:

print("Return only the number", fizz_buzz_encode(1))
print("Multiple of 3", fizz_buzz_encode(3))
print("Multiple of 5", fizz_buzz_encode(5))
print("Return only the number", fizz_buzz_encode(7))
print("Multiple of both 3 and 5", fizz_buzz_encode(15))# List out the state space. What are the different states that the data could be in:
# there's only 4 options/states in the FizzBuzz setup
# A number is a multiple of 3 and 5, only 3, only 5, or it ain't
# Now we'll need to one hot encode the training data for y
def fizz_buzz_encode(i):
    if i % 15 == 0:
        return np.array([0, 0, 0, 1]) # encoding for "fizzbuzz"
    elif i % 5 == 0: 
        return np.array([0, 0, 1, 0]) # encoding for "buzz"
    elif i % 3  == 0: 
        return np.array([0, 1, 0, 0]) # encoding for "fizz"
    else:
        return np.array([1, 0, 0, 0]) # encoding for the number output

In [16]:

print("Return only the number", fizz_buzz_encode(1))
print("Multiple of 3", fizz_buzz_encode(3))
print("Multiple of 5", fizz_buzz_encode(5))
print("Return only the number", fizz_buzz_encode(7))
print("Multiple of both 3 and 5", fizz_buzz_encode(15))

Return only the number [1 0 0 0]
Multiple of 3 [0 1 0 0]
Multiple of 5 [0 0 1 0]
Return only the number [1 0 0 0]
Multiple of both 3 and 5 [0 0 0 1]


In [17]:
# y_train is the encoded output. 
# This is our "labeled data" for supervised learning
y_train = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])
y_train

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0]])

In [18]:
# Now let's build our model, add layers, compile, and fit it!
model = Sequential()

In [19]:
## Add the model's layers. 
model.add(Dense(1000, input_dim=NUM_DIGITS, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(4, activation="softmax"))

In [20]:
# categorical cross entropy is the recommended loss function for 
# metrics is a good
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=["accuracy"])

In [21]:
model.fit(x_train, y_train, nb_epoch=20, batch_size=128)

Epoch 1/20
923/923 [==============================] - 0s 322us/step - loss: 1.7633 - accuracy: 0.4442
Epoch 2/20
923/923 [==============================] - 0s 238us/step - loss: 1.1445 - accuracy: 0.5341
Epoch 3/20
923/923 [==============================] - 0s 235us/step - loss: 1.1448 - accuracy: 0.5341
Epoch 4/20
923/923 [==============================] - 0s 241us/step - loss: 1.1412 - accuracy: 0.5341
Epoch 5/20
923/923 [==============================] - 0s 220us/step - loss: 1.1315 - accuracy: 0.5341
Epoch 6/20
923/923 [==============================] - 0s 205us/step - loss: 1.1229 - accuracy: 0.5341
Epoch 7/20
923/923 [==============================] - 0s 264us/step - loss: 1.1107 - accuracy: 0.5341
Epoch 8/20
923/923 [==============================] - 0s 223us/step - loss: 1.1012 - accuracy: 0.5352
Epoch 9/20
923/923 [==============================] - 0s 212us/step - loss: 1.0806 - accuracy: 0.5341
Epoch 10/20
923/923 [==============================] - 0s 323us/step - loss: 1.070

In [22]:
# convert fizzbuzz to binary
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [23]:
# Setup x_test  fizzbuzz for prime numbers from 1 to 100
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [24]:

# Setup predicted output
predictions = np.vectorize(fizz_buzz)(numbers, y_test)
print (predictions)

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' 'fizz' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' 'fizz' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' 'fizz' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' 'fizz' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100']


In [25]:

# correct answers for fizzbuzz on 1-100, these are our actual values
answer = np.array([])
for i in numbers:
    if i % 15 == 0: 
        answer = np.append(answer, "fizzbuzz")
    elif i % 5 == 0: 
        answer = np.append(answer, "buzz")
    elif i % 3 == 0: 
        answer = np.append(answer, "fizz")
    else: answer = np.append(answer, str(i))
print (answer)

['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' 'fizz' '22' '23' 'fizz'
 'buzz' '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz'
 'fizz' '37' '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47'
 'fizz' '49' 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59'
 'fizzbuzz' '61' '62' 'fizz' '64' 'buzz' 'fizz' '67' '68' 'fizz' 'buzz'
 '71' 'fizz' '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' 'fizz' '82'
 '83' 'fizz' 'buzz' '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94'
 'buzz' 'fizz' '97' '98' 'fizz' 'buzz']


In [26]:
# Let's evaluate the model's predictions
evaluate = np.array(answer == predictions)
print (np.count_nonzero(evaluate == True) / 100)

0.53


In [27]:
answer == predictions

array([ True,  True, False,  True, False, False,  True,  True, False,
       False,  True, False,  True,  True, False,  True,  True, False,
       False, False, False,  True,  True, False, False,  True, False,
        True,  True, False,  True,  True, False,  True, False, False,
        True,  True, False, False,  True, False,  True,  True, False,
        True,  True, False,  True, False,  True,  True,  True, False,
       False,  True, False,  True,  True, False,  True,  True, False,
        True, False, False,  True, False, False, False,  True, False,
        True,  True, False,  True,  True,  True,  True, False, False,
        True,  True, False, False,  True, False,  True,  True, False,
        True,  True, False,  True, False, False,  True,  True, False,
       False])